In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import PolynomialFeatures
! pip install pmdarima
from pmdarima.arima import auto_arima, ADFTest
from datetime import timedelta 

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 70)


In [3]:
df_data = pd.read_csv("owid-covid-data.csv")

In [8]:
df_data.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed',
       'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million',
       'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million',
       'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed',
       'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed', 'total_vaccinations_per_hundred',
   

In [15]:
cols_list = ['date','continent','location','new_cases_smoothed']
df_data_for_pred = df_data[cols_list].copy()
df_data_for_pred.head(20)

,date,continent,location,new_cases_smoothed
0,2020-02-24,Asia,Afghanistan,NaN
1,2020-02-25,Asia,Afghanistan,NaN
2,2020-02-26,Asia,Afghanistan,NaN
3,2020-02-27,Asia,Afghanistan,NaN
4,2020-02-28,Asia,Afghanistan,NaN
5,2020-02-29,Asia,Afghanistan,0.714
6,2020-03-01,Asia,Afghanistan,0.714
7,2020-03-02,Asia,Afghanistan,0.000
8,2020-03-03,Asia,Afghanistan,0.000
9,2020-03-04,Asia,Afghanistan,0.000


In [17]:
df_data_for_pred['new_cases_smoothed'].isnull().groupby(df_data_for_pred['location']).sum().to_dict()

{'Afghanistan': 7,
 'Africa': 5,
 'Albania': 18,
 'Algeria': 5,
 'Andorra': 5,
 'Angola': 9,
 'Anguilla': 5,
 'Antigua and Barbuda': 7,
 'Argentina': 72,
 'Armenia': 32,
 'Aruba': 5,
 'Asia': 5,
 'Australia': 7,
 'Austria': 6,
 'Azerbaijan': 6,
 'Bahamas': 6,
 'Bahrain': 5,
 'Bangladesh': 5,
 'Barbados': 5,
 'Belarus': 5,
 'Belgium': 6,
 'Belize': 5,
 'Benin': 6,
 'Bermuda': 7,
 'Bhutan': 5,
 'Bolivia': 5,
 'Bonaire Sint Eustatius and Saba': 5,
 'Bosnia and Herzegovina': 5,
 'Botswana': 8,
 'Brazil': 7,
 'British Virgin Islands': 5,
 'Brunei': 6,
 'Bulgaria': 5,
 'Burkina Faso': 5,
 'Burundi': 5,
 'Cambodia': 5,
 'Cameroon': 5,
 'Canada': 6,
 'Cape Verde': 5,
 'Cayman Islands': 9,
 'Central African Republic': 5,
 'Chad': 5,
 'Chile': 5,
 'China': 13,
 'Colombia': 6,
 'Comoros': 5,
 'Congo': 5,
 'Cook Islands': 5,
 'Costa Rica': 7,
 "Cote d'Ivoire": 6,
 'Croatia': 5,
 'Cuba': 5,
 'Curacao': 5,
 'Cyprus': 7,
 'Czechia': 7,
 'Democratic Republic of Congo': 6,
 'Denmark': 31,
 'Djibouti': 

In [18]:
null_data_2020 = df_data_for_pred[df_data_for_pred["date"]<='2020-12-31'].isnull().sum().values
null_data_2021 = df_data_for_pred[(df_data_for_pred["date"]>='2021-01-01') & (df_data_for_pred["date"]<='2021-12-31')].isnull().sum().values
null_data_2022 =df_data_for_pred[df_data_for_pred["date"]>='2022-01-01'].isnull().sum().values

df_null = pd.DataFrame(list(zip(null_data_2020,null_data_2021,null_data_2022)), columns=["2020","2021","2022"], index = df_data_for_pred.columns.to_list())
df_null

,2020,2021,2022
date,0,0,0
continent,4379,4745,2002
location,0,0,0
new_cases_smoothed,3017,4471,1441


In [19]:
# converts date to M8[ns], which is a 'synomym' to datetime64[ns]

df_data_for_pred['date']=pd.to_datetime(df_data_for_pred["date"],format='%Y-%m-%d')
df_data_for_pred["date"].dtypes

dtype('<M8[ns]')

In [23]:
countries_list = df_data_for_pred['location'].unique()
countries_list

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethi

In [30]:
interpolation_data_country = pd.DataFrame({"date":[],"country":[],"continent":[],
                         "new_cases_smoothed":[]})

interpolation_data_country

,date,country,continent,new_cases_smoothed


In [33]:
continent_list = df_data_for_pred['continent'].unique()
continent_list

array(['Asia', nan, 'Europe', 'Africa', 'North America', 'South America',
       'Oceania'], dtype=object)

In [46]:
europe_countries = df_data_for_pred[df_data_for_pred['continent'] == 'Europe'].copy()
europe_countries.head()

europe_countries_sorted = europe_countries.sort_values(["location", 'date'], ascending = [True, False]).copy()

europe_countries_sorted.head(5)

,date,continent,location,new_cases_smoothed
2502,2022-06-03,Europe,Albania,37.429
2501,2022-06-02,Europe,Albania,29.857
2500,2022-06-01,Europe,Albania,33.714
2499,2022-05-31,Europe,Albania,23.143
2498,2022-05-30,Europe,Albania,31.429


In [47]:
europe_country_list = europe_countries_sorted['location'].unique()

In [50]:
for country in europe_country_list:
    country_temp = europe_countries_sorted[europe_countries_sorted['location'] == country].copy()
    europe_countries_sorted["new_cases_smoothed"] = europe_countries_sorted["new_cases_smoothed"].interpolate(method = "linear",limit_direction = "both")

In [52]:
europe_countries_sorted.isnull().sum()

date                  0
continent             0
location              0
new_cases_smoothed    0
dtype: int64

In [53]:
# Sort our Dataframe by location in ascending order, and by date in Ascending order
europe_countries_sorted = europe_countries_sorted.sort_values(["location", 'date'], ascending = [True, True]).copy()
europe_countries_sorted.head(5)

,date,continent,location,new_cases_smoothed
1673,2020-02-25,Europe,Albania,23.699
1674,2020-02-26,Europe,Albania,22.684
1675,2020-02-27,Europe,Albania,21.669
1676,2020-02-28,Europe,Albania,20.654
1677,2020-02-29,Europe,Albania,19.639


In [73]:
# drop the continent column

europe_countries_sorted.drop("continent", axis=1, inplace=True)

In [58]:
# Create an empty dataframe

Prediction_per_country = pd.DataFrame({"date":[],"Country":[],
                         "new_cases":[]})
Prediction_per_country

,date,Country,new cases


In [60]:
countries_list = europe_countries_sorted['location'].unique()

for country in countries_list:
    print(country)
    
    model_sarima = auto_arima(europe_countries_sorted[europe_countries_sorted['location'] == country]['new_cases_smoothed'], trace = True, error_action = 'ignore', 
                         start_p = 0, start_q = 0, max_p = 2, max_q = 2, m = 7,
                   suppress_warnings = True, stepwise = True, seasonal = True)
    model_sarima.fit(europe_countries_sorted[europe_countries_sorted['location'] == country]['new_cases_smoothed'])
    
    # Prediction for country for the next 365 days
    prediction_sarima = model_sarima.predict(365)
    
    # Getting the last Date that the country has data
    temp_country = europe_countries_sorted[europe_countries_sorted['location'] == country].copy()
    max_date_country = temp_country["date"].max()
    new_date_country = max_date_country + pd.Timedelta(days=1)
    
    for i in range(len(prediction_sarima)):
        Prediction_per_country.loc[len(Prediction_per_country.index)] = [new_date_country,country, prediction_sarima[i]] 
        new_date_country = new_date_country + pd.Timedelta(days=1)

Albania
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=inf, Time=0.92 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=8788.286, Time=0.03 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=8735.954, Time=0.36 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=8723.379, Time=0.41 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=8786.286, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[7] intercept   : AIC=8790.273, Time=0.08 sec
 ARIMA(0,1,1)(1,0,1)[7] intercept   : AIC=inf, Time=1.41 sec
 ARIMA(0,1,1)(0,0,2)[7] intercept   : AIC=8715.858, Time=0.83 sec
 ARIMA(0,1,1)(1,0,2)[7] intercept   : AIC=inf, Time=1.27 sec
 ARIMA(0,1,0)(0,0,2)[7] intercept   : AIC=8728.422, Time=0.44 sec
 ARIMA(1,1,1)(0,0,2)[7] intercept   : AIC=inf, Time=1.41 sec
 ARIMA(0,1,2)(0,0,2)[7] intercept   : AIC=8575.354, Time=0.99 sec
 ARIMA(0,1,2)(0,0,1)[7] intercept   : AIC=8581.464, Time=0.64 sec
 ARIMA(0,1,2)(1,0,2)[7] intercept   : AIC=inf, Time=2.74 sec
 ARIMA(0,1,2)(1,0,1)[7] intercept   : AIC=inf, Tim

 ARIMA(0,1,2)(2,0,0)[7]             : AIC=8828.120, Time=0.93 sec
 ARIMA(1,1,1)(2,0,0)[7]             : AIC=8401.720, Time=0.57 sec
 ARIMA(2,1,2)(2,0,0)[7]             : AIC=8400.579, Time=1.35 sec
 ARIMA(0,1,1)(2,0,0)[7]             : AIC=9104.810, Time=0.45 sec
 ARIMA(2,1,1)(2,0,0)[7]             : AIC=8400.701, Time=1.03 sec

Best model:  ARIMA(1,1,2)(2,0,0)[7]          
Total fit time: 45.950 seconds
Belgium
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=13962.962, Time=0.31 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=13965.908, Time=0.03 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=13960.475, Time=0.18 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=13961.412, Time=0.21 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=13963.911, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[7] intercept   : AIC=13959.560, Time=0.06 sec
 ARIMA(1,1,0)(0,0,1)[7] intercept   : AIC=13960.139, Time=0.21 sec
 ARIMA(1,1,0)(1,0,1)[7] intercept   : AIC=13960.562, Time=0.53 sec
 ARIM

 ARIMA(2,1,2)(0,0,1)[7] intercept   : AIC=9530.035, Time=2.08 sec
 ARIMA(2,1,2)(0,0,2)[7] intercept   : AIC=9568.740, Time=3.13 sec
 ARIMA(2,1,2)(1,0,2)[7] intercept   : AIC=inf, Time=3.53 sec
 ARIMA(1,1,2)(0,0,1)[7] intercept   : AIC=9565.562, Time=1.13 sec
 ARIMA(1,1,1)(0,0,1)[7] intercept   : AIC=9568.908, Time=0.83 sec
 ARIMA(2,1,2)(0,0,1)[7]             : AIC=9528.036, Time=1.20 sec
 ARIMA(2,1,2)(0,0,0)[7]             : AIC=9607.442, Time=0.47 sec
 ARIMA(2,1,2)(1,0,1)[7]             : AIC=inf, Time=2.11 sec
 ARIMA(2,1,2)(0,0,2)[7]             : AIC=9566.556, Time=1.88 sec
 ARIMA(2,1,2)(1,0,0)[7]             : AIC=9529.798, Time=1.23 sec
 ARIMA(2,1,2)(1,0,2)[7]             : AIC=inf, Time=2.76 sec
 ARIMA(1,1,2)(0,0,1)[7]             : AIC=9563.565, Time=0.59 sec
 ARIMA(2,1,1)(0,0,1)[7]             : AIC=9565.407, Time=0.65 sec
 ARIMA(1,1,1)(0,0,1)[7]             : AIC=9566.910, Time=0.57 sec

Best model:  ARIMA(2,1,2)(0,0,1)[7]          
Total fit time: 58.531 seconds
Croatia
Perfo

 ARIMA(2,1,0)(1,0,2)[7]             : AIC=inf, Time=2.38 sec
 ARIMA(1,1,1)(1,0,2)[7]             : AIC=inf, Time=2.79 sec
 ARIMA(0,1,1)(1,0,2)[7]             : AIC=inf, Time=1.64 sec
 ARIMA(2,1,1)(1,0,2)[7]             : AIC=inf, Time=3.35 sec

Best model:  ARIMA(1,1,0)(1,0,2)[7]          
Total fit time: 45.759 seconds
Denmark
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=13052.181, Time=0.36 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=13055.412, Time=0.04 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=12811.725, Time=0.66 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=12949.124, Time=0.73 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=13053.414, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[7] intercept   : AIC=12924.109, Time=0.08 sec
 ARIMA(1,1,0)(2,0,0)[7] intercept   : AIC=12771.627, Time=1.23 sec
 ARIMA(1,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=2.63 sec
 ARIMA(1,1,0)(1,0,1)[7] intercept   : AIC=inf, Time=1.75 sec
 ARIMA(0,1,0)(2,0,0)[7] intercept   

 ARIMA(2,1,0)(2,0,2)[7] intercept   : AIC=12793.375, Time=2.93 sec
 ARIMA(2,1,0)(1,0,2)[7] intercept   : AIC=inf, Time=2.44 sec
 ARIMA(2,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=2.53 sec
 ARIMA(2,1,0)(1,0,1)[7] intercept   : AIC=12801.657, Time=1.09 sec
 ARIMA(2,1,1)(2,0,2)[7] intercept   : AIC=inf, Time=3.58 sec
 ARIMA(1,1,1)(2,0,2)[7] intercept   : AIC=inf, Time=3.08 sec
 ARIMA(2,1,0)(2,0,2)[7]             : AIC=inf, Time=1.72 sec

Best model:  ARIMA(2,1,0)(2,0,2)[7] intercept
Total fit time: 32.602 seconds
France
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=16935.051, Time=0.33 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=16946.080, Time=0.04 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=16878.512, Time=0.26 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=16920.106, Time=0.22 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=16944.084, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[7] intercept   : AIC=16895.206, Time=0.09 sec
 ARIMA(1,1,0)(2,0,0)[7] interce

...

 ARIMA(2,1,0)(0,0,2)[7] intercept   : AIC=11999.310, Time=0.95 sec
 ARIMA(2,1,0)(2,0,0)[7] intercept   : AIC=12003.153, Time=1.98 sec
 ARIMA(2,1,0)(2,0,2)[7] intercept   : AIC=inf, Time=3.12 sec
 ARIMA(2,1,1)(1,0,1)[7] intercept   : AIC=12000.709, Time=2.15 sec
 ARIMA(1,1,1)(1,0,1)[7] intercept   : AIC=11995.959, Time=1.67 sec
 ARIMA(1,1,1)(0,0,1)[7] intercept   : AIC=11996.632, Time=1.02 sec
 ARIMA(1,1,1)(1,0,0)[7] intercept   : AIC=12001.945, Time=0.73 sec
 ARIMA(1,1,1)(2,0,1)[7] intercept   : AIC=inf, Time=2.77 sec
 ARIMA(1,1,1)(1,0,2)[7] intercept   : AIC=inf, Time=2.89 sec
 ARIMA(1,1,1)(0,0,0)[7] intercept   : AIC=12062.808, Time=0.57 sec
 ARIMA(1,1,1)(0,0,2)[7] intercept   : AIC=11998.221, Time=1.77 sec
 ARIMA(1,1,1)(2,0,0)[7] intercept   : AIC=12002.423, Time=1.81 sec
 ARIMA(1,1,1)(2,0,2)[7] intercept   : AIC=inf, Time=2.79 sec
 ARIMA(0,1,1)(1,0,1)[7] intercept   : AIC=12475.042, Time=0.76 sec
 ARIMA(1,1,2)(1,0,1)[7] intercept   : AIC=11981.427, Time=1.92 sec
 ARIMA(1,1,2)(0,0,1

 ARIMA(2,1,0)(0,0,0)[7] intercept   : AIC=11623.057, Time=0.31 sec
 ARIMA(2,1,0)(2,0,0)[7] intercept   : AIC=11621.351, Time=1.14 sec
 ARIMA(2,1,0)(1,0,1)[7] intercept   : AIC=11617.730, Time=1.87 sec
 ARIMA(2,1,0)(0,0,1)[7] intercept   : AIC=11619.501, Time=0.45 sec
 ARIMA(2,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=4.02 sec
 ARIMA(2,1,0)(1,0,2)[7] intercept   : AIC=inf, Time=4.07 sec
 ARIMA(2,1,0)(0,0,2)[7] intercept   : AIC=11620.633, Time=0.93 sec
 ARIMA(2,1,0)(2,0,2)[7] intercept   : AIC=inf, Time=3.47 sec
 ARIMA(2,1,1)(1,0,1)[7] intercept   : AIC=11500.072, Time=1.72 sec
 ARIMA(2,1,1)(0,0,1)[7] intercept   : AIC=11499.862, Time=1.37 sec
 ARIMA(2,1,1)(0,0,0)[7] intercept   : AIC=11503.029, Time=0.65 sec
 ARIMA(2,1,1)(0,0,2)[7] intercept   : AIC=11500.467, Time=2.48 sec
 ARIMA(2,1,1)(1,0,0)[7] intercept   : AIC=11500.037, Time=1.56 sec
 ARIMA(2,1,1)(1,0,2)[7] intercept   : AIC=inf, Time=3.33 sec
 ARIMA(1,1,1)(0,0,1)[7] intercept   : AIC=11671.303, Time=0.95 sec
 ARIMA(2,1,2)(0,0,1

 ARIMA(2,1,0)(1,0,0)[7] intercept   : AIC=11119.403, Time=0.83 sec
 ARIMA(2,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=2.44 sec
 ARIMA(2,1,0)(1,0,1)[7] intercept   : AIC=11077.298, Time=1.62 sec
 ARIMA(2,1,0)(0,0,1)[7] intercept   : AIC=11094.549, Time=0.63 sec
 ARIMA(2,1,0)(1,0,2)[7] intercept   : AIC=inf, Time=2.44 sec
 ARIMA(2,1,0)(0,0,0)[7] intercept   : AIC=11157.909, Time=0.13 sec
 ARIMA(2,1,0)(0,0,2)[7] intercept   : AIC=11082.329, Time=1.28 sec
 ARIMA(2,1,0)(2,0,2)[7] intercept   : AIC=inf, Time=2.65 sec
 ARIMA(2,1,1)(1,0,1)[7] intercept   : AIC=inf, Time=2.07 sec
 ARIMA(1,1,1)(1,0,1)[7] intercept   : AIC=inf, Time=1.32 sec
 ARIMA(2,1,0)(1,0,1)[7]             : AIC=11075.310, Time=0.73 sec
 ARIMA(2,1,0)(0,0,1)[7]             : AIC=11092.549, Time=0.37 sec
 ARIMA(2,1,0)(1,0,0)[7]             : AIC=11117.403, Time=0.36 sec
 ARIMA(2,1,0)(2,0,1)[7]             : AIC=inf, Time=2.20 sec
 ARIMA(2,1,0)(1,0,2)[7]             : AIC=inf, Time=1.93 sec
 ARIMA(2,1,0)(0,0,0)[7]             :

 ARIMA(1,1,2)(2,0,0)[7] intercept   : AIC=13306.413, Time=2.50 sec
 ARIMA(2,1,2)(2,0,0)[7]             : AIC=13302.756, Time=1.79 sec
 ARIMA(2,1,2)(1,0,0)[7]             : AIC=13308.146, Time=1.11 sec
 ARIMA(2,1,2)(2,0,1)[7]             : AIC=inf, Time=3.37 sec
 ARIMA(2,1,2)(1,0,1)[7]             : AIC=13305.382, Time=1.48 sec
 ARIMA(1,1,2)(2,0,0)[7]             : AIC=13304.414, Time=1.04 sec
 ARIMA(2,1,1)(2,0,0)[7]             : AIC=13305.139, Time=1.10 sec
 ARIMA(1,1,1)(2,0,0)[7]             : AIC=13311.415, Time=0.79 sec

Best model:  ARIMA(2,1,2)(2,0,0)[7]          
Total fit time: 45.448 seconds
Switzerland
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=12726.890, Time=0.56 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=12790.544, Time=0.03 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=12704.594, Time=0.66 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=12715.523, Time=0.62 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=12788.546, Time=0.02 sec
 A

 ARIMA(1,1,0)(0,0,0)[7] intercept   : AIC=-1845.699, Time=0.20 sec
 ARIMA(1,1,0)(2,0,0)[7] intercept   : AIC=-2168.444, Time=1.00 sec
 ARIMA(1,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=2.58 sec
 ARIMA(1,1,0)(1,0,1)[7] intercept   : AIC=inf, Time=1.28 sec
 ARIMA(0,1,0)(2,0,0)[7] intercept   : AIC=-2170.262, Time=0.77 sec
 ARIMA(0,1,0)(1,0,0)[7] intercept   : AIC=-2077.983, Time=0.46 sec
 ARIMA(0,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=2.04 sec
 ARIMA(0,1,1)(2,0,0)[7] intercept   : AIC=-2168.426, Time=0.92 sec
 ARIMA(1,1,1)(2,0,0)[7] intercept   : AIC=-2166.484, Time=2.20 sec
 ARIMA(0,1,0)(2,0,0)[7]             : AIC=-2172.255, Time=0.52 sec
 ARIMA(0,1,0)(1,0,0)[7]             : AIC=-2079.978, Time=0.16 sec
 ARIMA(0,1,0)(2,0,1)[7]             : AIC=inf, Time=0.92 sec
 ARIMA(0,1,0)(1,0,1)[7]             : AIC=inf, Time=0.53 sec
 ARIMA(1,1,0)(2,0,0)[7]             : AIC=-2170.438, Time=0.46 sec
 ARIMA(0,1,1)(2,0,0)[7]             : AIC=-2170.420, Time=0.53 sec
 ARIMA(1,1,1)(2,0,0)[7]  

In [61]:
Prediction_per_country.head()

,date,Country,new cases
0,2022-06-04,Albania,38.884742
1,2022-06-05,Albania,44.671107
2,2022-06-06,Albania,47.513952
3,2022-06-07,Albania,53.511098
4,2022-06-08,Albania,51.772155


In [67]:
# rename columns to match the original dataset columns

Prediction_per_country.rename(columns = {'Country':'location', 'new cases':'new_cases_smoothed'}, inplace = True)
Prediction_per_country.head()

,date,location,new_cases_smoothed
0,2022-06-04,Albania,38.884742
1,2022-06-05,Albania,44.671107
2,2022-06-06,Albania,47.513952
3,2022-06-07,Albania,53.511098
4,2022-06-08,Albania,51.772155


In [75]:
# combine the original dataframe with the predictions dataframe

combined_country_data = pd.concat([europe_countries_sorted, Prediction_per_country], ignore_index=True)
combined_country_data.shape

(61757, 3)

In [78]:
combined_country_data_sorted = combined_country_data.sort_values(["location", 'date'], ascending = [True, True]).copy()

In [79]:
combined_country_data.to_csv(r"C:\Users\Home1\Desktop\Rhoe Project\combined_country_data.csv", index = False) 